In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import csv

import gaiaxpy

In [38]:
def convert_csv(file_path):
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        with open(f'data-53-txt/{file_path[10:-4]}.txt', 'w') as txt_file:
            for row in csv_reader:
                txt_file.write(' '.join(row) + '\n')

In [39]:
directory = './data-53'

def get_unique_target_ids():
    arr = []
    for filename in os.listdir(directory):
        arr.append(filename.rsplit('_')[0])
    return list(set(arr))

target_ids = get_unique_target_ids()
target_ids

['22745910577134848',
 '418001477279154304',
 '3704392873141718912',
 '6749419923164242816',
 '984190381489031424',
 '4237169341286788096',
 '3302846072717868416',
 '45980377978968064',
 '5444093531284061568',
 '2405805697263561600',
 '3079329274484276480',
 '2633603478379307904',
 '3772994667772794112',
 '2901155648586891648',
 '6903096911032851584',
 '3944400490365194368',
 '4249667902270614272',
 '3289827339649866880',
 '266077145295627520',
 '2448177130188111232',
 '3918811315733520640',
 '3348071631670500736',
 '3629615946329315456',
 '3935488605023787392',
 '4452521234885949184',
 '1473687671071803520',
 '2375647158466154112',
 '6210089815971933056',
 '3652142603120146560',
 '6018034958869558912',
 '4922382445088509312',
 '2398029465700538880',
 '6709878766312642944',
 '1510374147844219904',
 '4646535078125821568',
 '781164326766404736',
 '3304090857318319232',
 '775029739076581248',
 '6127333286605955072',
 '3959631234670040704',
 '4016474027321256704',
 '3587107658569000960',
 

In [40]:
data_dict = {}

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        identifier, file_type = filename.rsplit('_', 1)[0], filename.rsplit('_', 1)[1].split('.')[0]

        # Read the file contents
        file_path = os.path.join(directory, filename)
        convert_csv(file_path)

In [46]:
def parse_file(file_path):
    entities = []
    current_entity = {'id': None, 'array1': [], 'array2': []}
    current_array = None  # Will hold which array we are currently filling ('array1' or 'array2')

    with open(file_path, 'r') as f:
        next(f)  # Skip the first line
        for line in f:
            line = line.strip()

            # Case 1: New ID found
            if line.isdigit():
                # Save the previous entity if it's complete
                if current_entity['id'] is not None:
                    entities.append(current_entity.copy())

                # Reset current_entity and current_array
                current_entity = {'id': line, 'array1': [], 'array2': []}
                current_array = 'array1'

            # Case 2: Line is part of an array
            elif current_array:  # Ensure current_array is not None
                # Remove brackets and split the line into numbers
                numbers = list(map(int, line.replace('[', '').replace(']', '').split()))

                # Extend the current array with these numbers
                current_entity[current_array].extend(numbers)

                # Switch to the second array if the first array is closed
                if line.endswith(']'):
                    current_array = 'array2'

    # Don't forget to add the last entity
    if current_entity['id'] is not None:
        entities.append(current_entity.copy())

    return entities

In [49]:
# metadata parsing
parsed_dir = 'data-53-txt/spdata'
targets_spectra = []


all_targets = {}

all_entities = {}
# Display the first few entities from the first file to check if everything is working as expected

for filename in os.listdir(parsed_dir):
    identifier, file_type = filename.rsplit('_', 1)[0], filename.rsplit('_', 1)[1].split('.')[0]

    # Read the file contents
    file_path = os.path.join(parsed_dir, filename)
    target = parse_file(file_path)
    all_entities[file_path] = target

print(all_entities)

{'data-53-txt/spdata/3918811315733520640_spdata.txt': [], 'data-53-txt/spdata/6903096911032851584_spdata.txt': [], 'data-53-txt/spdata/1510374147844219904_spdata.txt': [], 'data-53-txt/spdata/4646535078125821568_spdata.txt': [], 'data-53-txt/spdata/4452521234885949184_spdata.txt': [], 'data-53-txt/spdata/418001477279154304_spdata.txt': [], 'data-53-txt/spdata/3652142603120146560_spdata.txt': [], 'data-53-txt/spdata/4855735169813450624_spdata.txt': [], 'data-53-txt/spdata/266077145295627520_spdata.txt': [], 'data-53-txt/spdata/781164326766404736_spdata.txt': [], 'data-53-txt/spdata/3935488605023787392_spdata.txt': [], 'data-53-txt/spdata/2633603478379307904_spdata.txt': [], 'data-53-txt/spdata/3079329274484276480_spdata.txt': [], 'data-53-txt/spdata/3302846072717868416_spdata.txt': [], 'data-53-txt/spdata/6018034958869558912_spdata.txt': [], 'data-53-txt/spdata/2814629409239942272_spdata.txt': [], 'data-53-txt/spdata/3772994667772794112_spdata.txt': [], 'data-53-txt/spdata/7925889397736

- #transitid: Identifier for the transit, usually a unique number.
- obstime: Observation time, usually in ISO 8601 format.
- fluxbpint: Integrated flux in the blue photometric band.
- fluxbpinterror: Error in the integrated flux in the blue photometric band.
- fluxrpint: Integrated flux in the red photometric band.
- fluxrpinterror: Error in the integrated flux in the red photometric band.
- bpimagequality: Quality metric for the image in the blue photometric band.
- rpimagequality: Quality metric for the image in the red photometric band.
- nu: Frequency of the observed phenomena, usually in Hz.
- dnu: Error in the frequency.
- flagspe: Flag related to the spectral data; usually an integer where each bit may represent some condition or quality metric.
- provenance: Identifier to trace back the data, often an internal code.
- run: Run identifier, typically used for data processing.
- apflags: Flags related to aperture photometry.
- obsobmt: Likely another observation time metric, possibly in a different format or reference frame.
- bpwinal, bpwinac, rpwinal, rpwinac: Likely related to the window used for the blue and red photometric bands, possibly indicating the size and orientation of the window.

In [52]:
read_csv("./data-53/22745910577134848_spdata.csv")[1]

['103806349721006833',
 '[ 19049  19072  19231  19356  19526  19767  20163  20475  20807  21266']

In [48]:
test_data = read_csv("./data-53/22745910577134848_spdata.csv")

for n in range(1,len(test_data)):
    if n % 11 == 0:
        print(test_data[1][n])

IndexError: list index out of range